<a href="https://colab.research.google.com/github/DiegoLangreo7/EII-RI/blob/main/Ejercicio5_RI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ej. 5. Comparación con búsqueda semántica**

En este nuevo ejercicio, se nos pide que comparemos el rendimiento de la mejor configuración de BM25 con expansión de consultas del cuarto ejercicio (es decir, la que obtuvimos de turnar distintos valores de n con m) contra un sistema de recuperación semántico que emplee una base de datos vectorial y un modelo pre-entrenado de word embeddings—puedes utilizar el del notebook visto en los laboratorios, ademas de realizar un análisis comparativo detallado, discutiendo las fortalezas y debilidades de cada enfoque para diferentes tipos de consulta.

Para este ejercicio nos apoyaremos en el tercer guion de practicas, el cual trata este tema.

Lo primero de todo que haremos sera descargarnos los paquetes para utilizar ChromaDB, e importarlos, que es lo usado en practicas y lo que uso de referencia:

In [ ]:
!pip install chromadb
!pip install sentence_transformers

import chromadb
import json
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

Luego nos descargamos nuevamente nuestra coleccion Trec-Covid y posteriormente, lo descomprimimos y parseamos lisa-corpus.json:

In [ ]:
# Descargamos la colección y las consultas y las query
!gdown 19pzNFYIch8rj9d3kyq-171V8vRa_wLBC
!unzip -o trec-covid-RI.zip

# Parseamos la colección
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_content = []
    for line in f:
        corpus_content.append(json.loads(line))

Para crear la colección, necesitamos instanciar un cliente, lo cual haremos como en practicas:

In [ ]:
# Initialize the sentence transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Create a persistent ChromaDB client
client = chromadb.PersistentClient(path="./chromadb-storage/")

# We create the collection, please note how we are providing the embedding
# pre-trained model (this is a multilingual model) and we specify the
# distance metric to find the nearest neighbors
collection = client.create_collection(
  name="TREC-COVID",
  embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"),
  metadata={"hnsw:space": "cosine"}
)

Ahora preparamos los documentos:

In [ ]:
chromadb_documents = []
chromadb_doc_ids = []

for document in corpus_content:
  doc_id = str(document["_id"])
  title = document["title"].lower()
  content = document["text"].lower()

  chromadb_doc_ids.append(doc_id)
  chromadb_documents.append(f"{title} {content}")


# Con esto crearemos los embeddings:
chromadb_embeddings = model.encode(chromadb_documents, batch_size=100, show_progress_bar=True, device='cuda')

Ahora que tenemos los embeddings de los documentos, podemos comenzar a añadir documentos, identificadores y embeddings a la colección en ChromaDB:

In [ ]:
# En primer lugar, definimos una función para generar lotes:
def get_batches(lista, chunk_size=100):
    return [lista[i:i + chunk_size] for i in range(0, len(lista), chunk_size)]

In [ ]:
# A continuación, utilizamos la función anterior para ir creando la colección por lotes:
document_batches = get_batches(chromadb_documents)
ids_batches = get_batches(chromadb_doc_ids)
embedding_batches = get_batches(chromadb_embeddings)

for i in range(len(document_batches)):
  documents = document_batches[i]
  doc_ids = ids_batches[i]
  embeddings = embedding_batches[i]

  collection.add(
    documents=documents,
    ids=doc_ids,
    embeddings=embeddings
  )

Una vez tenemos la coleccion creada, vamos a proseguir a buscar con chromaDB:


In [ ]:
# Crear un cliente persistente de ChromaDB
client = chromadb.PersistentClient(path="./chromadb-storage/")

# Obtener las colecciones disponibles en ChromaDB
existing_collections = client.list_collections()

collection_name = "TREC-COVID"

# Inicializar el modelo transformador de oraciones
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Verificar si la colección existe
if collection_name in [col.name for col in existing_collections]:
    # Tiene poco sentido que sea necesario especificar qué función de embeddings usa la
    # colección *pero* si no se informa explícitamente,
    # Chroma utilizará la función de embeddings predeterminada y será como
    # comparar peras con manzanas...
    collection = client.get_collection(
        collection_name,
        embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    )

    existing_ids = collection.get()["ids"]
    print(f"La colección {collection_name} contiene {len(existing_ids)} documentos")
else:
    print(f"¡{collection_name} no existe! Es necesario crearla.")


Ahora es necesario cargar las consultas, para ello reutilizaremos codigo de anteriores ejercicios:

In [ ]:
# Queries cargadas
queries = []

with open("queries.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        queries.append(json.loads(line))

Vamos a crear una función para enviar las consultas y obtener los resultados asociados a ellas, para
posteriormente calcular el rendimiento (precisión, exhaustividad y F1-score).

In [ ]:
# Función para cargar consultas desde un archivo JSON para enviarlas a una colección
# y obtener listas de resultados
def submit_queries_and_get_run(queries, collection, max_results=10):
    # Inicializar el diccionario de ejecuciones (run)
    run = {}

    # Procesar cada consulta
    for query in queries:
        query_id = query["_id"]
        query_text = query["text"].lower()

        # Enviar la consulta a la colección y obtener los resultados
        results = collection.query(
            query_texts=[query_text],
            n_results=max_results
        )

        # Almacenar los IDs de los resultados en el diccionario de ejecuciones
        run[query_id] = results['ids'][0]

    return run

Estas son las consultas originales en la colección de evaluación; fueron recopiladas por los autores de la
colección, por lo que el desajuste de vocabulario entre las consultas y los documentos debería ser mínimo.

In [ ]:
original_run = submit_queries_and_get_run(queries, collection)

Ahora definimos el codigo para calcular la precisión y la exhaustividad y F1-score

In [ ]:
# Función para calcular precisión, recall y F1-score tanto en promedio micro como macro
def compute_precision_recall_f1(run, relevance_judgements):
    # Inicializar listas para almacenar los valores de precisión, recall y F1 para cada consulta
    precision_values = []
    recall_values = []
    f1_values = []

    # Inicializar conteos globales para el promedio micro
    global_retrieved = 0
    global_relevant = 0
    global_retrieved_and_relevant = 0

    # Calcular precisión, recall y F1-score para cada consulta
    for query_id in run.keys():
        retrieved_results = run[query_id]
        relevant_results = relevance_judgements[query_id]
        relevant_and_retrieved = set(retrieved_results) & set(relevant_results)

        # Actualizar conteos globales
        global_retrieved += len(retrieved_results)
        global_relevant += len(relevant_results)
        global_retrieved_and_relevant += len(relevant_and_retrieved)

        # Calcular precisión y recall
        precision = len(relevant_and_retrieved) / len(retrieved_results) if len(retrieved_results) > 0 else 0
        recall = len(relevant_and_retrieved) / len(relevant_results) if len(relevant_results) > 0 else 0

        # Calcular F1-score si ambos precisión y recall son mayores a cero
        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_values.append(f1)

        # Agregar precisión y recall de la consulta actual
        precision_values.append(precision)
        recall_values.append(recall)

    # Calcular promedios macro
    macro_average_precision = sum(precision_values) / len(precision_values) if precision_values else 0
    macro_average_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    macro_average_f1 = sum(f1_values) / len(f1_values) if f1_values else 0

    # Imprimir métricas promediadas macro
    print(f"Precisión promediada macro: {round(macro_average_precision,3)}")
    print(f"Recall promediado macro: {round(macro_average_recall,3)}")
    print(f"F1 promediado macro: {round(macro_average_f1,3)}")
    print("")

    # Calcular promedios micro
    micro_average_precision = global_retrieved_and_relevant / global_retrieved if global_retrieved > 0 else 0
    micro_average_recall = global_retrieved_and_relevant / global_relevant if global_relevant > 0 else 0
    micro_average_f1 = (2 * (micro_average_precision * micro_average_recall) /
                        (micro_average_precision + micro_average_recall)) if (micro_average_precision + micro_average_recall) > 0 else 0

    # Imprimir métricas promediadas micro
    print(f"Precisión promediada micro: {round(micro_average_precision,3)}")
    print(f"Recall promediado micro: {round(micro_average_recall,3)}")
    print(f"F1 promediado micro: {round(micro_average_f1,3)}")


Tambien tenemos que cargar los juicios, los cuales los cargaremos como hasta ahora, aunque sera necesaria una modificacion:

In [ ]:
# Cargamos los juicios de relevancia desde qrels.tsv
relevance_judgements = {}
with open("qrels.tsv", "r", encoding="utf-8") as f:
    next(f)  # Saltar la cabecera
    for line in f:
        query_id, corpus_id, score = line.strip().split("\t")
        if int(score) > 0:  # Considerar solo documentos relevantes (score > 0)
            if query_id not in relevance_judgements:
                relevance_judgements[query_id] = []
            relevance_judgements[query_id].append(corpus_id)

# Formatear los juicios de relevancia
relevance_judgements_reformat = {}
for query_id, rel_docs in relevance_judgements.items():
    relevance_judgements_reformat[query_id] = rel_docs

# Imprimir el resultado formateado
print(relevance_judgements_reformat)


Con todo lo anterior ya podemos comprobar que tan bueno es la busqueda con este procedimiento:

In [ ]:
compute_precision_recall_f1(original_run, relevance_judgements_reformat)